In [2]:
import sklearn
import pandas as pd
import numpy as np
from IPython.display import display
import functools as ft
import matplotlib.pyplot as plt

In [3]:
# Features: https://drive.google.com/file/d/1GM0V4BPFLfYbohwnh8L1DelbM8_n-OvU/view
# Time: https://drive.google.com/file/d/14gLfWcZBm0JdBUnrwtkjNBpgLmRT0LYA/view
# Label: https://drive.google.com/file/d/1G6IE9ldfykeiT-7C514uhoK1z_VZZnFI/view
# Flow: https://drive.google.com/file/d/1gqESUaxOjAGxGIuDV1QB_C56aIOJv8GM/view
# Content: https://drive.google.com/file/d/16tgrJRwwBXiLFPYevh9FOBn_Abip25jp/view
# Basic: https://drive.google.com/file/d/1nrC-44vn-sHaV-ZW8EYEy2YjZxiIUvbg/view
    # Additional: https://drive.google.com/file/d/1fl7ZyrVVWh-pm2L5hnuIs4GU4N-a0G6X/view

time = pd.read_csv('https://drive.google.com/uc?id=14gLfWcZBm0JdBUnrwtkjNBpgLmRT0LYA')
label = pd.read_csv('https://drive.google.com/uc?id=1G6IE9ldfykeiT-7C514uhoK1z_VZZnFI')
flow = pd.read_csv('https://drive.google.com/uc?id=1gqESUaxOjAGxGIuDV1QB_C56aIOJv8GM')
content = pd.read_csv('https://drive.google.com/uc?id=16tgrJRwwBXiLFPYevh9FOBn_Abip25jp')
basic = pd.read_csv('https://drive.google.com/uc?id=1nrC-44vn-sHaV-ZW8EYEy2YjZxiIUvbg')
additional = pd.read_csv('https://drive.google.com/uc?id=1fl7ZyrVVWh-pm2L5hnuIs4GU4N-a0G6X')

feats = pd.read_csv('https://drive.google.com/uc?id=1GM0V4BPFLfYbohwnh8L1DelbM8_n-OvU')
df = ft.reduce(lambda left, right: pd.merge(left, right, on='id'), [time, label, flow, content, basic, additional])


In [4]:
df_train = df[:len(df)//2]
df_eval = df[len(df)//2:]

# display(df_train)
display(df_eval)

,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat,id,attack_cat,label,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
87670,4135.305056,4364.118183,78.840667,50.610849,0.129734,0.052500,0.077234,87671,Exploits,1,...,0,0,0,1,1,1,1,1,1,1
87671,4105.506280,154.478813,73.919459,100.062219,0.103869,0.059652,0.044217,87672,Fuzzers,1,...,0,0,0,2,2,1,1,1,1,3
87672,3757.980897,158.143313,73.346366,108.471570,0.125644,0.063001,0.062643,87673,Fuzzers,1,...,0,0,0,2,2,2,2,2,1,2
87673,695518.386200,22964.600000,7076.945100,8845.795000,0.103796,0.069486,0.034310,87674,Fuzzers,1,...,0,0,0,3,11,3,2,2,2,3
87674,2851.046694,115.278992,56.020090,71.994289,0.180173,0.089726,0.090447,87675,Fuzzers,1,...,0,0,0,4,4,1,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,0.000000,175337,Generic,1,...,0,0,0,24,24,24,24,24,13,24
175337,3721.068786,120.177727,54.400111,66.980570,0.099440,0.036895,0.062545,175338,Shellcode,1,...,0,0,0,1,1,1,1,1,1,2
175338,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,0.000000,175339,Generic,1,...,0,0,0,12,12,3,3,3,3,13
175339,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,0.000000,175340,Generic,1,...,0,0,0,30,30,30,30,30,14,30


In [5]:
from math import log2
from collections import Counter

def calcentropy(array: list[any]) -> float :
    return sum(- (x := v/len(array) ) *log2(x) for v in Counter(array).values() )

def calcentropy_fromdict(di : dict, el_cnt) -> float :
    ans = 0
    for v in di.values() :
        if v : 
            x = v / el_cnt
            try :
                ans += -x*log2(x)
            except :
                print(x, "math domain error")
    return ans

def calcgain(dataframe, gainCol, targetCol) :
    vals = list(dataframe[gainCol].unique())
    dfdict = {v : dataframe[dataframe[gainCol] == v] for v in vals}
    assert(sum(len(d) for d in dfdict.values() ) == len(dataframe))
    en_S = calcentropy(dataframe[targetCol])
    return en_S - sum(
        calcentropy(dfdict[v][targetCol]) * len(dfdict[v]) / len(dataframe)
        for v in vals
    )



# calcgain(df, 'service', 'attack_cat')

testdf = pd.DataFrame({
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
})

# example gain in ppt
if __name__ == "__main__" :
    print ("Outlook:", calcgain(testdf, 'Outlook', 'PlayTennis') )
    print ("Humidity:" , calcgain(testdf, 'Humidity', 'PlayTennis') )
    print ("Temperature:" ,calcgain(testdf, 'Temperature', 'PlayTennis') )
    print ("Wind:" ,calcgain(testdf, 'Wind', 'PlayTennis') )

Outlook: 0.24674981977443922
Humidity: 0.15183550136234159
Temperature: 0.02922256565895487
Wind: 0.04812703040826949


In [6]:
from pandas import DataFrame
from collections import defaultdict
T = lambda: defaultdict(T)

result = 0

def plurality(examples: DataFrame, targetCol) :
    return examples[targetCol].mode()[0]
    
def make_id3_tree(cur: DataFrame, attr: list[str], prev: DataFrame, targetCol, maxdepth = 10, depth = 0) :

    if prev is None :
        print("Creating tree. Hang on, this might take a while.")
        print("Each dot printed represents a tree node has been created.")
    
    root = T()
    print(".", end = '')
    if cur.empty: 
        root[result] = plurality(prev, targetCol)
    elif len(cur[targetCol].unique()) == 1 :
        root[result] = cur[targetCol].unique()[0]
    elif not attr or maxdepth == depth :
        root[result] = plurality(cur, targetCol)

    else :
        A = max(attr, key = lambda at: calcgain(dataframe=cur, gainCol=at, targetCol=targetCol))
        for vk in cur[A].unique() :
            nxt = cur[cur[A] == vk]
            nxt_attr = attr.copy()
            nxt_attr.remove(A)
            subtree = make_id3_tree(nxt, nxt_attr, cur, targetCol, maxdepth, depth + 1)
            root[(A, vk)] = subtree

    if prev is None : print("Done!")
    return root

def id3classify(tree: T, data: dict) :
    
    while result not in tree :
        found = 0
        for key, subtree in tree.items() :
            
            attr, attr_val = key
            if data[attr] == attr_val :
                tree = subtree
                found = 1
                break

        if not found :
            tree = tree[next(iter(tree.keys()))]
            
    return tree[result]

In [7]:
attr = set(testdf.columns)
targetCol = "PlayTennis"
attr.remove(targetCol)
TEST_TREE = make_id3_tree(testdf, attr, None, targetCol)

Creating tree. Hang on, this might take a while.
Each dot printed represents a tree node has been created.
........Done!


In [8]:
def print_tree(t, indent = 0) :
    for k,v in t.items() :
        print(f'{indent * " "}')
        if isinstance(v, defaultdict) :
            attr, attr_val = k
            print(f'{" "*indent}> ({attr} = {attr_val})')
            print_tree(v, indent + 2)
            
        else :
            print(f'{" "*indent}> ', end= ' ')
            print(v)

print_tree(TEST_TREE)


> (Outlook = Sunny)
  
  > (Humidity = High)
    
    >  No
  
  > (Humidity = Normal)
    
    >  Yes

> (Outlook = Overcast)
  
  >  Yes

> (Outlook = Rain)
  
  > (Wind = Weak)
    
    >  Yes
  
  > (Wind = Strong)
    
    >  No


In [ ]:
# def get_column_breakoffs(dataframe, targetCol):
#     breakoffs = {}
#     en_S = calcentropy(dataframe[targetCol])

#     for idx, k in enumerate(dataframe.columns):
#         if dataframe[k].dtype in ["float64", "int64"]:
#             mx = 0
#             best = 727  # wysi
#             dataframe.sort_values(by=k, inplace=True)
#             Lcnt = defaultdict(int)
#             Rcnt = Counter(dataframe[targetCol])

#             for i in range(len(dataframe) - 1):
#                 Lcnt[dataframe[targetCol].iloc[i]] += 1
#                 Rcnt[dataframe[targetCol].iloc[i]] -= 1
                
#                 cur = en_S - (
#                     calcentropy_fromdict(Lcnt, i + 1) * ((i + 1) / len(dataframe)) + 
#                     calcentropy_fromdict(Rcnt, len(dataframe) - i - 1) * (1 - ((i + 1) / len(dataframe)))
#                 )

#                 if cur > mx:
#                     mx = cur
#                     best = i
            
#             breakoffs[k] = dataframe[k].iloc[best]

#         print(k, "breakoff computed!")
    
#     return breakoffs


    

# def discretize_df(dataframe, breakoffs):
#     for k, breakoff in breakoffs.items():
#         if dataframe[k].dtype in ["float64", "int64"]:
#             dataframe.loc[dataframe[k] < breakoff, k] = -1
#             dataframe.loc[dataframe[k] >= breakoff, k] = 1
#     return dataframe
    
# breakoffs = get_column_breakoffs(df_train, 'attack_cat')
# df_train = discretize_df(df_train, breakoffs)

In [9]:
# test df

ttime = pd.read_csv('test/time_features_test.csv')
tflow = pd.read_csv('test/flow_features_test.csv')
tcontent = pd.read_csv('test/content_features_test.csv')
tbasic = pd.read_csv('test/basic_features_test.csv')
tadditional = pd.read_csv('test/additional_features_test.csv')

tdf = ft.reduce(lambda left, right: pd.merge(left, right, on='id'), [ttime, tflow, tcontent, tbasic, tadditional])
tdf

,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat,id,proto,swin,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,2737.954123,118.833969,48.756556,76.593602,0.165117,0.072001,0.093116,0,tcp,255.0,...,0.0,0.0,0.0,5.0,5.0,2.0,2.0,2.0,1.0,2.0
1,2938.299144,165.780563,49.812539,109.557602,0.223604,0.100248,0.123356,1,tcp,255.0,...,0.0,NaN,0.0,6.0,6.0,1.0,1.0,1.0,1.0,5.0
2,4287.453629,129.471406,69.765530,94.395906,0.113189,0.082498,0.030691,2,tcp,255.0,...,0.0,0.0,0.0,4.0,4.0,1.0,2.0,1.0,1.0,4.0
3,0.000000,0.000000,0.001000,0.000000,0.000000,0.000000,0.000000,3,udp,0.0,...,0.0,0.0,0.0,10.0,4.0,2.0,4.0,2.0,1.0,4.0
4,1119.063538,26.748141,17.628799,15.543294,0.000655,0.000526,0.000129,4,tcp,255.0,...,NaN,0.0,0.0,13.0,11.0,10.0,7.0,6.0,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20578,1902.551328,143.722203,16.088482,87.730055,0.117474,0.071119,0.046355,20578,tcp,255.0,...,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0
20579,858.139465,22.118752,14.832222,13.803000,0.000647,0.000491,0.000156,20579,tcp,255.0,...,0.0,0.0,0.0,14.0,7.0,1.0,9.0,1.0,1.0,1.0
20580,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,20580,udp,0.0,...,0.0,0.0,0.0,6.0,6.0,NaN,4.0,4.0,4.0,4.0
20581,0.000000,0.000000,0.011000,0.000000,0.000000,0.000000,0.000000,20581,unas,0.0,...,0.0,0.0,0.0,11.0,11.0,4.0,8.0,4.0,4.0,4.0


In [25]:
attr = set(df_train.columns)
targetCol = "attack_cat"
attr.remove(targetCol)
for k in df_train.columns :
    if df_train[k].dtype in ["float64", "int64"]:
        if len(df_train[k].unique()) > 30 :
            attr.remove(k)

for x in ['id', 'label'] :
    if x in attr :
        attr.remove(x)

TREE = make_id3_tree(df_train, attr, None, targetCol, maxdepth = 8)

{'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'state', 'proto', 'sttl', 'ct_flw_http_mthd', 'dttl', 'trans_depth', 'service', 'is_ftp_login', 'dwin', 'swin', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_ftp_cmd'}
Creating tree. Hang on, this might take a while.
Each dot printed represents a tree node has been created.
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [26]:

hit = 0
for i in range(len(df_eval)) :
    data = df_eval.iloc[i].to_dict()
    res = id3classify(TREE, data)
    hit += data['attack_cat'] == res

acc = hit / len(df_eval)
acc

0.2620706961252866